# CÁLCULO DE VIGA PROTENDIDA - VP11

**OBRA:** Carnetti Open Mall

**Cliente:** JD Construções e Pré-Fabricados

**Data:** 18/10/2022

**Cálculo:** Gabriel Garcia

<img src="img/logoRS.jpg" alt="RS Engenharia & Arquitetura" width="200px"/>

Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [1]:
import handcalcs.render
import pandas as pd
from math import floor,ceil,sqrt,exp
from numpy import log
from ggcp import calcpsi, linha_neutra_cp_ret
    
pi = 3.141592653

# Informações Iniciais:

## Definição do EPS:

In [2]:
%%render

h_eps = 40
e_eps = 12

<IPython.core.display.Latex object>

## Geometria da Viga:

In [3]:
%%render
# Geometria da Viga:

h = 90
b = 30
Area = b*h - e_eps*h_eps
Inercia = b*h**3/12 - e_eps*(h_eps**3)/12
W_i = Inercia/(h/2)
W_s = Inercia/(h/2)
Vao = 1964

## Número de seções para dividir o vão:
# Escolher número impar
nS = 15

<IPython.core.display.Latex object>

## Propriedades dos Materiais:

In [4]:
%%render
# Propriedades dos Materiais:

f_ck = 4
f_ctm = 0.03*(10*f_ck)**(2/3)
f_ptk = 190
f_pyd = 148.7
f_yk = 50
f_cj = 2.8

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [5]:
%%render 2 short

f_c = f_ck*0.85/1.4
E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

## Carregamentos:

In [6]:
%%render 2 short

Q_t0 = 0.000025*Area
Q_tF = Q_t0 + (50+50+50)*6/100/100


<IPython.core.display.Latex object>

## Esforços no Centro do Vão:

In [7]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
V_F = (Q_tF)*Vao/2

<IPython.core.display.Latex object>

# Deteminação da Força de Protensão:

## Limites de Tensão:

In [8]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

## Cálculo da Força de Protensão:

In [9]:
%%render short 2

e_p = 35

P_1 = (sigma_cc0+M_0/W_i)/(r_0*(1/Area + e_p/W_i)) 
P_2 = (sigma_ct0-M_0/W_s)/(r_0*(1/Area - e_p/W_s)) 
P_3 = (sigma_ctF+M_F/W_i)/(r_F*(1/Area + e_p/W_i))
P_4 = (sigma_ccF-M_F/W_s)/(r_F*(1/Area - e_p/W_s))

##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## Escolha da Protensão:

In [10]:
%%render 2
# Força de Protensão Adotada:

P = P_max
phi_1 = 1.52
A_p1 = 1.43

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## Ancoragem das Cordoalhas:

### Comprimento de Ancoragem Básico:

In [11]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### Cálculo do comprimento de Transferência:

In [12]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.0 * l_bpt2

l_p = (h**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  Esforços nas Seções:

## Momento Fletor e Cortante:

In [13]:
esforcos = []
esforcos_t0 = []
for i in range(3):
    esforcos.append( [0]*(nS) )
    esforcos_t0.append( [0]*(nS) )
    
# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(nS):
    esforcos[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos[1][i] = round( 0.5*Q_tF*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos[2][i] = round( 0.5*Q_tF*(Vao-2*esforcos[0][i]), 1 )
    esforcos_t0[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos_t0[1][i] = round( 0.5*Q_t0*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos_t0[2][i] = round( 0.5*Q_t0*(Vao-2*esforcos[0][i]), 1 )
    
    
tabela = pd.DataFrame(esforcos)
tabela = tabela.transpose()
tabela.rename(columns={0:'Pos (cm)',1:'M (kNcm)',2:'V (kN)'},inplace = True)
display(tabela)

,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,142.9
1,140.3,18614.2,122.5
2,280.6,34364.3,102.1
3,420.9,47250.5,81.6
4,561.1,57266.4,61.2
5,701.4,64426.5,40.8
6,841.7,68722.6,20.4
7,982.0,70154.6,0.0
8,1122.3,68722.6,-20.4
9,1262.6,64426.5,-40.8


# Escalonamento da Protensão:

## Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

In [14]:
%%render short

M_s1 = round( 0.5*Q_t0*esforcos[0][1]*(Vao-esforcos[0][1]) ,1) # Momento de peso próprio na seção s1
P_1 = (sigma_cc0+M_s1/W_i)/(r_0*(1/Area + e_p/W_i))
P_2 = (sigma_ct0-M_s1/W_s)/(r_0*(1/Area - e_p/W_s))


<IPython.core.display.Latex object>

In [15]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## Escolha do Número de Cordoalhas:

In [16]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = [4,6]

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [35,33.333]

<IPython.core.display.Latex object>

# Esforços Devidos À Protensão:

In [17]:
A_p = []

for i in range(len(qntCordoalhas)):
    A_p.append(qntCordoalhas[i] * A_p1)

W_cp = []

for i in range(len(qntCordoalhas)):
    W_cp.append(round(Inercia/e_p_i[i],0))
    
alpha_p0 = E_p / E_cst0

K_p0 = []
for i in range(len(qntCordoalhas)):
    K_p0.append(round(alpha_p0*A_p[i]*(1/Area+e_p_i[i]/W_cp[i]),4))
    
sigma_p = []
for i in range(len(qntCordoalhas)):
    sigma_p.append(round(sigma_p0/(1+K_p0[i]),2))
    
N_p = []
for i in range(len(qntCordoalhas)):
    N_p.append(round(-sigma_p[i]*A_p[i],1))
    
M_p = []
for i in range(len(qntCordoalhas)):
    M_p.append(round(N_p[i]*e_p_i[i],1))
    
tabela1 = pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1 = tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma,p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,"Sigma,p",Np,Mp
0,5.72,50243.0,0.0509,144.64,-827.3,-28955.5
1,8.58,52756.0,0.0720,141.79,-1216.6,-40552.9


# Escalonamento da Força de Protensão:

In [18]:
M_g = []
x1 = []
x2 = []
x1_anc = []
x2_anc = []

for i in range(len(qntCordoalhas)):
    M_g.append(round( W_s*( (N_p[i]/Area) -(M_p[i]/W_s) + sigma_ct0),2) )
    #print("M_g["+str(i)+"]:",M_g[i])
        
for i in range(len(qntCordoalhas)):
    # O erro acontece no caso da raiz de delta retornar um número complexo
    try:
        c = 2*M_g[i]/Q_t0
        delta = (-Vao)**2 - 4*c
        #print("Delta "+str(i)+":",round(delta,2))
        x1.append(round( (Vao-delta**0.5)/2  ,2))
        x2.append(round( (Vao+delta**0.5)/2  ,2))
    except:
        print("##   ATENÇÃO!   ##")
        raise Exception("ERRO: ESCALONAR MELHOR A PROTENSÃO!")    

for i in range(len(qntCordoalhas)):
    x1_anc.append(x1[i]-round(l_bpt,2))
    x2_anc.append(x2[i]+round(l_bpt,2))
    if x1_anc[i] < 0:
        x1_anc[i] = 0
    if x2_anc[i] > Vao:
        x2_anc[i] = Vao
    
tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,1420.94,26.43,1937.57,0.0,1964.0
1,6165.65,120.53,1843.47,0.0,1964.0


# Perdas Progressivas da Força de Protensão:

## Fluência do Concreto:

### Fluência Rápida Irreversível

In [19]:
%%render params

# VER ITEM 12.3.3 da NBR6118:2014
# Valores para adotar:
# s = 0.38 para concreto de cimento CP-III e CP-IV
# s = 0.25 para concreto de cimento CP-I e CP-II
# s = 0.20 para concreto de cimento CP-V
s = 0.20
t_0 = 10
t_F = 10000


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))

<IPython.core.display.Latex object>

In [20]:
if s == 0.38:
    print("Adotado s = 0.38 para concreto de cimento CP-III ou CP-IV")
elif s == 0.25:
    print("Adotado s = 0.25 para concreto de cimento CP-I ou CP-II")
else:
    print("Adotado s = 0.20 para concreto de cimento CP-V (ARI) ")

Adotado s = 0.20 para concreto de cimento CP-V (ARI) 


### Fluência Lenta Irreversível:

In [21]:
%%render params 2

U = 55 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h + 2*b )

h_fic = gamma*(2*Area)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### Fluência Lenta Reversível:

In [22]:
%%render params

phi_dF = 0.4

beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### Fluência Total:

In [23]:
%%render 2 params
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

### Retração do Concreto:

In [24]:
%%render 2 params

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)


<IPython.core.display.Latex object>

## Relaxação do Aço de Protensão:

In [25]:
relax = []
for i in range(7):
    relax.append([0]*nS)

for i in range (nS):
    relax[0][i] = int(i+1)
    relax[1][i] = esforcos[0][i] #posição x da seção
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            relax[2][i] = round(sigma_p[j],2)
            break
    relax[3][i] = round(relax[2][i]/f_ptk,3)
    relax[4][i] = round(calcpsi(relax[3][i]),3)
    relax[5][i] = round(relax[4][i]*2.5,3)
    relax[6][i] = round(-log(1-relax[5][i]),3)
    
    
tabela3 = pd.DataFrame(relax)
tabela3 = tabela3.transpose()
tabela3.rename(columns={0:'Seção',1:'Pos',2:'Sigma,p',3:'Sigma,p/fptk',4:'\u03A81000',5:'\u03A8,inf',6:'X,inf'},inplace = True)
display(tabela3)

,Seção,Pos,"Sigma,p","Sigma,p/fptk",Ψ1000,"Ψ,inf","X,inf"
0,1.0,0.0,141.79,0.746,0.03,0.075,0.078
1,2.0,140.3,141.79,0.746,0.03,0.075,0.078
2,3.0,280.6,141.79,0.746,0.03,0.075,0.078
3,4.0,420.9,141.79,0.746,0.03,0.075,0.078
4,5.0,561.1,141.79,0.746,0.03,0.075,0.078
5,6.0,701.4,141.79,0.746,0.03,0.075,0.078
6,7.0,841.7,141.79,0.746,0.03,0.075,0.078
7,8.0,982.0,141.79,0.746,0.03,0.075,0.078
8,9.0,1122.3,141.79,0.746,0.03,0.075,0.078
9,10.0,1262.6,141.79,0.746,0.03,0.075,0.078


## Cálculo das Perdas Progressivas:

In [26]:
progressivas = []

for i in range(4):
    progressivas.append([0] * nS)
    
alpha_p = E_p/E_cs
    
for i in range(nS):
    pos_i = relax[1][i]
    momento = 0.5*Q_t0*pos_i*(Vao-pos_i)
    #momento = esforcos[1][i]
    #print("Momento seção "+str(i)+": "+str(int(momento)))
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    progressivas[0][i] = round(Npi/Area + Mpi/Wcpi + momento/Wcpi,2)
    #print(Npi,Mpi)
    K_p = alpha_p*A_pp*(1/Area + e_pp/Wcpi)
    progressivas[1][i] = round( (( progressivas[0][i]*phi_t/E_cs + epsilon_cs - sigma_pi/E_p*relax[6][i] ) / (1 + relax[6][i] + K_p*(1+0.5*phi_t)) )*E_p,2)
    progressivas[2][i] = -round(progressivas[1][i]*A_pp  ,2)
    progressivas[3][i] = round(progressivas[2][i]*e_pp ,1)
    
    

tabela4 = pd.DataFrame(progressivas)
tabela4 = tabela4.transpose()
tabela4.rename(columns={0:'Sigma,c,t0',1:'\u0394Sigma,p',2:'\u0394Np',3:'\u0394Mp'},inplace = True)
display(tabela4)

,"Sigma,c,t0","ΔSigma,p",ΔNp,ΔMp
0,-1.32,-39.96,342.86,11428.6
1,-1.18,-37.45,321.32,10710.6
2,-1.07,-35.48,304.42,10147.2
3,-0.98,-33.86,290.52,9683.9
4,-0.90,-32.43,278.25,9274.9
5,-0.85,-31.53,270.53,9017.6
6,-0.82,-30.99,265.89,8862.9
7,-0.81,-30.82,264.44,8814.6
8,-0.82,-30.99,265.89,8862.9
9,-0.85,-31.53,270.53,9017.6


# Tensões Normais nas Bordas:

## Bordas Superiores:

In [27]:
tensaosup = []

for i in range(3):
    tensaosup.append([0]*nS)
    
for i in range(nS):
    tensaosup[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaosup[1][i] = round( -abs(Npx/Area) + abs(Mpx/W_s) - abs(esforcos[1][i]/W_s) ,3)
    if tensaosup[1][i] > -sigma_ccF:
        tensaosup[2][i] = "OK"
    else:
        tensaosup[2][i] = "--> NAO OK <--"

tabela5 = pd.DataFrame(tensaosup)
tabela5 = tabela5.transpose()
tabela5.rename(columns={0:'Pos (cm)',1:'Sigma,s',2:'OK'},inplace = True)
display(tabela5)

,Pos (cm),"Sigma,s",OK
0,0.0,0.352,OK
1,140.3,-0.116,OK
2,280.6,-0.512,OK
3,420.9,-0.836,OK
4,561.1,-1.088,OK
5,701.4,-1.268,OK
6,841.7,-1.376,OK
7,982.0,-1.412,OK
8,1122.3,-1.376,OK
9,1262.6,-1.268,OK


## Bordas Inferiores:

In [28]:
tensaoinf = []

for i in range(3):
    tensaoinf.append([0]*nS)
    
for i in range(nS):
    tensaoinf[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaoinf[1][i] = round( -abs(Npx/Area) - abs(Mpx/W_i) + abs(esforcos[1][i]/W_i) ,3)
    if tensaoinf[1][i] < -sigma_ctF:
        tensaoinf[2][i] = "OK"
    else:
        tensaoinf[2][i] = "--> NAO OK <--"
    
tabela6 = pd.DataFrame(tensaoinf)
tabela6 = tabela6.transpose()
tabela6.rename(columns={0:'Pos (cm)',1:'Sigma,i',2:'OK'},inplace = True)
display(tabela6)

,Pos (cm),"Sigma,i",OK
0,0.0,-1.139,OK
1,140.3,-0.691,OK
2,280.6,-0.31,OK
3,420.9,0.002,OK
4,561.1,0.242,OK
5,701.4,0.416,OK
6,841.7,0.519,--> NAO OK <--
7,982.0,0.554,--> NAO OK <--
8,1122.3,0.519,--> NAO OK <--
9,1262.6,0.416,OK


# Armaduras Passivas:

## Armadura da Cunha de Tração:

In [29]:
sigma_cs0 = -10000
pos = -1
for i in range (0,nS):
    if sigma_cs0 < tensaosup[1][i]:
        sigma_cs0 = tensaosup[1][i]
        pos = i

sigma_ci0 = tensaoinf[1][pos]

print("Seção crítica de cálculo: seção",pos)

Seção crítica de cálculo: seção 0


In [30]:
%%render 2

x = sigma_cs0/(sigma_cs0 - sigma_ci0) * h

S_x = b*x**2/2

R_ct = sigma_cs0/x*S_x

A_st = R_ct/25.00

phi_st = 1.6

N_st = ceil(A_st/((phi_st/2)**2*pi))

<IPython.core.display.Latex object>

## Pré-Alongamento de Leito:

In [31]:
prealong = []

for i in range(5):
    prealong.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
                sigma_pw = sigma_p[j]
                Npw = N_p[j]
                Mpw = M_p[j]
                Wcpw = W_cp[j]
                break
    prealong[0][i] = Npw + progressivas[2][i]
    prealong[1][i] = Mpw + progressivas[3][i]
    prealong[2][i] = round(prealong[0][i]/Area + prealong[1][i]/Wcpw ,2)
    prealong[3][i] = round( sigma_pw + progressivas[1][i] - alpha_p*prealong[2][i] ,2)
    prealong[4][i] = round(prealong[3][i]/E_p,5)

tabela7 = pd.DataFrame(prealong)
tabela7 = tabela7.transpose()
tabela7.rename(columns={0:'Np,F',1:'Mp,F',2:'Sigma,cpF', 3:'Sigma,PN',4:'e,pn'},inplace = True)
display(tabela7)

,"Np,F","Mp,F","Sigma,cpF","Sigma,PN","e,pn"
0,-873.74,-29124.3,-0.95,107.79,0.00539
1,-895.28,-29842.3,-0.97,110.43,0.00552
2,-912.18,-30405.7,-0.99,112.52,0.00563
3,-926.08,-30869.0,-1.00,114.20,0.00571
4,-938.35,-31278.0,-1.02,115.76,0.00579
5,-946.07,-31535.3,-1.02,116.66,0.00583
6,-950.71,-31690.0,-1.03,117.26,0.00586
7,-952.16,-31738.3,-1.03,117.43,0.00587
8,-950.71,-31690.0,-1.03,117.26,0.00586
9,-946.07,-31535.3,-1.02,116.66,0.00583


## Armadura Passiva de Flexão:

![formulario](img/formulario.png)

In [32]:
d_p = [] # braço de alavanca da armadura ativa
d_s = [] # braço de alavanca da armadura passiva

for i in range(len(qntCordoalhas)):
    d_p.append(e_p_i[i] + h/2)
    d_s.append(h-5) # estimativa para ds
    
A_s = []
for i in range(8):
    A_s.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pm = sigma_p[j]
            Npm = N_p[j]
            Mpm = M_p[j]
            Wcpm = W_cp[j]
            A_pm = A_p[j]
            d_sm = d_s[j]
            d_pm = d_p[j]
            break
    Md = esforcos[1][i]*1.4
    retorno = linha_neutra_cp_ret(b,h,f_c,A_pm,prealong[4][i],d_pm,d_sm,Md)
    A_s[0][i] = int(Md)
    A_s[1][i] = retorno[0]
    A_s[2][i] = round(retorno[0]/d_sm,3)
    A_s[3][i] = retorno[2]
    A_s[5][i] = retorno[1]
    A_s[4][i] = retorno[4]
    A_s[6][i] = retorno[5]
    A_s[7][i] = round((0.8*b*f_c*retorno[0]-retorno[3])/retorno[4],2)
    

tabela8 = pd.DataFrame(A_s)
tabela8 = tabela8.transpose()
tabela8.rename(columns={0:'Md', 1:'x',2:'x/d',3:'eps,s',4:'sigma,s',5:'eps,p',6:'sigma,p',7:'As'},inplace = True)
display(tabela8)

,Md,x,x/d,"eps,s","sigma,s","eps,p","sigma,p",As
0,0.0,1.73,0.020,0.16836,43.48,0.15974,148.7,-27.02
1,26059.0,7.22,0.085,0.03769,43.48,0.03943,148.7,-19.66
2,48110.0,12.12,0.143,0.02105,43.48,0.02419,148.7,-13.10
3,66150.0,16.32,0.192,0.01473,43.48,0.01844,148.7,-7.47
4,80172.0,19.73,0.232,0.01158,43.48,0.01561,148.7,-2.89
5,90197.0,22.25,0.262,0.00987,43.48,0.01407,148.7,0.48
6,96211.0,23.80,0.280,0.00900,43.48,0.01329,148.7,2.56
7,98216.0,24.33,0.286,0.00873,43.48,0.01305,148.7,3.27
8,96211.0,23.80,0.280,0.00900,43.48,0.01329,148.7,2.56
9,90197.0,22.25,0.262,0.00987,43.48,0.01407,148.7,0.48


# Dimensionamento ao Esforço Cortante:

## Verificação da Diagonal Comprimida:

In [33]:
%%render

V_d = V_F * 1.4
tau_wd = V_d/((b-e_eps)*d_sm)
tau_wu = 0.27*(1-f_ck/25)*f_ck/1.4


<IPython.core.display.Latex object>

In [34]:
if tau_wu > tau_wd:
    print("OK!")
else:
    print("NÃO OK!")

OK!


## Cálculo da Armadura de Corte:

### Armadura Mínima:

In [35]:
%%render short

rho_min = 0.2*f_ctm/f_yk
A_sws = rho_min*b

s = 15

A_sw_min = A_sws*s

nr = 2

phi_min = 2*(A_sw_min/(pi*nr))**0.5


<IPython.core.display.Latex object>

### Armadura Calculada:

In [36]:
i = int((nS+1)/2)
M_pf = prealong[1][i]
N_pf = prealong[0][i]

In [37]:
%%render

sigma_ciF = N_pf/Area + M_pf/W_i
M_0 = -sigma_ciF*W_i
M_d_max = 1.4*M_F

V_c0 = 0.6*f_ctd*b*d_sm
V_c = (1+M_0/M_d_max)*V_c0

V_sw = 1.4*V_F - V_c

A_sws = V_sw/(0.9*d_sm*f_yd)

s = 20
nr = 2
phi_min = 2*(A_sws/(pi*nr))**0.5

<IPython.core.display.Latex object>

## Armadura de Introdução da Força de Protensão:

In [38]:
# VERIFICAR SE ESTA É MESMO A FORÇA DE PROTENSÃO INSERIDA NO APOIO

N_p1 = -N_p[0]

In [39]:
%%render

A_swi = (0.25*N_p1)/25.00

<IPython.core.display.Latex object>

Esta armadura deve ser distribuída num trecho equivalente à 70% do comprimento de regularização.

In [40]:
%%render

l_d = l_p*0.7

<IPython.core.display.Latex object>

# Cálculo da Flecha Elástica:

**Flecha devida à protensão:**

Ap = área de aço de protensão

b = ponto de início da atuação da força

Delta = flecha causada por cada conjunto de cordoalhas

In [41]:

delta_p = []

delta_pF = 0

for i in range(len(qntCordoalhas)):
    delta_p.append([0] * 3)
    delta_p[i][0] = A_p[i]
    delta_p[i][1] = x1_anc[i]
    
    if(i != 0):
        delta_p[i][0] = delta_p[i][0] - delta_p[i-1][0]
        Momen = M_p[i] - M_p[i-1]
    else:
        Momen = M_p[i]
    
    if(delta_p[i][1] == 0):
        delta_p[i][2] = round(Momen*Vao**2/(8*E_cs*Inercia),2)
    else:
        delta_p[i][1] = (x1_anc[i]+x1[i])/2
        delta_p[i][2] = round( (Momen*(Vao**2-3*delta_p[i][1]**2-(Vao/2)**2)*(Vao/2))
                             /(6*E_cs*Inercia*Vao) ,2)
        
for i in range(len(qntCordoalhas)):
    delta_pF += delta_p[i][2]
    
tabela9 = pd.DataFrame(delta_p)
tabela9.rename(columns={0:'Ap',1:'b',2:'Delta'},inplace = True)

display(tabela9)

if (delta_p[0][0]) < 0:
    raise Exception("ERRO")

,Ap,b,Delta
0,5.72,0,-2.49
1,2.86,0,-1.00


In [42]:
%%render
delta_pF

limite = Vao/250

delta_i = 5/384 * (Q_tF*Vao**4)/(Inercia*E_cs)

delta_0 = 5/384 * (Q_t0*Vao**4)/(Inercia*E_cs)

delta_ti = delta_pF + delta_i

delta_t0 = delta_pF + delta_0

delta_tF = 2.5*delta_ti

<IPython.core.display.Latex object>